In [1]:
import numpy as np
import numpy
import pandas as pd
from sksurv.metrics import concordance_index_censored, brier_score, cumulative_dynamic_auc
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

from scipy import stats
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve,f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import ShuffleSplit, GridSearchCV
from skopt import BayesSearchCV

In [2]:
x_train1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Xtrain_0820.csv')
y_train1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Ytrain_competing_risk_0727.csv')
x_test1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Xtest_0820.csv')
y_test1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Ytest_competing_risk_0816_10y.csv')

In [3]:
x_train1_grs = x_train1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo","SCORE",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]
x_test1_grs = x_test1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo","SCORE",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]

In [4]:
x_train1_nogrs = x_train1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]
x_test1_nogrs = x_test1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]

In [5]:
y_train1_array_h = y_train1[["mof","mofDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_h]
y_train1_array_rsf_h = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_h = y_test1[["mof","mofDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_test1_array_h]
y_test1_array_rsf_h = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_train1_array_d = y_train1[["Death_10y","DeathDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_d]
y_train1_array_rsf_d = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_d = y_test1[["Death_10y","DeathDAY"]].to_numpy()
#List of tuples
aux = [(e1,e2) for e1,e2 in y_test1_array_d]
y_test1_array_rsf_d = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_train1_array_cr = y_train1[["mof_cr","mofDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_cr]
y_train1_array_rsf_cr = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_cr = y_test1[["mof_cr","mofDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_test1_array_cr]
y_test1_array_rsf_cr = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

In [6]:
# Model 4 (Bayesian optimization + FRAX CRFs + GRS)
estimator_h = ComponentwiseGradientBoostingSurvivalAnalysis(
    n_estimators = 152,
    learning_rate = 0.8503841871781402,
    subsample = 0.20019232109029358,
    random_state=0)

gcv_h = estimator_h.fit(x_train1_grs, y_train1_array_rsf_h)
gcv_d = ComponentwiseGradientBoostingSurvivalAnalysis().fit(x_train1_grs,y_train1_array_rsf_d)

cindex_h = concordance_index_censored(
    y_test1_array_rsf_cr["Status"],
    y_test1_array_rsf_cr["Survival_in_days"],
    gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs)
)
brier_h = brier_score(
    y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs),  
    times=3649
)
auc, mean_auc = cumulative_dynamic_auc(y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs),
    times=3649)

print("Model 4: ")
print("C-index: ", cindex_h[0])
print("Brier score: ", brier_h[1])
print("Dynamic mean auc: ", mean_auc)

Model 4: 
C-index:  0.7957865608323563
Brier score:  [0.93640552]
Dynamic mean auc:  0.7992079420655325


In [7]:
# Model 3 (grid search + FRAX CRFs + GRS)
estimator_h = ComponentwiseGradientBoostingSurvivalAnalysis(
    n_estimators = 250,
    learning_rate = 1,
    subsample = 0.2,
    random_state=0)

gcv_h = estimator_h.fit(x_train1_grs, y_train1_array_rsf_h)
gcv_d = ComponentwiseGradientBoostingSurvivalAnalysis().fit(x_train1_grs,y_train1_array_rsf_d)

cindex_h = concordance_index_censored(
    y_test1_array_rsf_cr["Status"],
    y_test1_array_rsf_cr["Survival_in_days"],
    gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs)
)
brier_h = brier_score(
    y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs),  
    times=3649
)
auc, mean_auc = cumulative_dynamic_auc(y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs),
    times=3649)
print("Model 3: ")
print("C-index: ", cindex_h[0])
print("Brier score: ", brier_h[1])
print("Dynamic mean auc: ", mean_auc)

Model 3: 
C-index:  0.78070294175129
Brier score:  [0.92501]
Dynamic mean auc:  0.7841135586203506


In [8]:
# Model 2 (Bayesian optimization + FRAX CRFs + GRS)
estimator_h = ComponentwiseGradientBoostingSurvivalAnalysis(
    n_estimators = 171,
    learning_rate = 0.15594561383166003,
    subsample = 0.4899995215939888,
    random_state=0)

gcv_h = estimator_h.fit(x_train1_nogrs, y_train1_array_rsf_h)
gcv_d = ComponentwiseGradientBoostingSurvivalAnalysis().fit(x_train1_nogrs,y_train1_array_rsf_d)

cindex_h = concordance_index_censored(
    y_test1_array_rsf_cr["Status"],
    y_test1_array_rsf_cr["Survival_in_days"],
    gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),
)
brier_h = brier_score(
    y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),  
    times=3649)
auc, mean_auc = cumulative_dynamic_auc(y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),
    times=3649)
print("Model 2: ")
print("C-index: ", cindex_h[0])
print("Brier score: ", brier_h[1])
print("Dynamic mean auc: ", mean_auc)

Model 2: 
C-index:  0.76900137570377
Brier score:  [0.93980281]
Dynamic mean auc:  0.7746722120161946


In [9]:
# Model 1 (grid search + FRAX CRFs + GRS)
estimator_h = ComponentwiseGradientBoostingSurvivalAnalysis(
    n_estimators = 250,
    learning_rate = 1,
    subsample = 0.5,
    random_state=0)

gcv_h = estimator_h.fit(x_train1_nogrs, y_train1_array_rsf_h)
gcv_d = ComponentwiseGradientBoostingSurvivalAnalysis().fit(x_train1_nogrs,y_train1_array_rsf_d)

cindex_h = concordance_index_censored(
    y_test1_array_rsf_cr["Status"],
    y_test1_array_rsf_cr["Survival_in_days"],
    gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),
)
brier_h = brier_score(
    y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),  
    times=3649)
auc, mean_auc = cumulative_dynamic_auc(y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),
    times=3649)
print("Model 1: ")
print("C-index: ", cindex_h[0])
print("Brier score: ", brier_h[1])
print("Dynamic mean auc: ", mean_auc)

Model 1: 
C-index:  0.7701920618144301
Brier score:  [0.93917]
Dynamic mean auc:  0.7745901500091
